# total Risk applied



In [3]:

!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

spark = SparkSession.builder.appName("TotalRiskEstimation").getOrCreate()

# Load the data (replace with your actual path)
df = spark.read.csv("/content/fb_price.csv", header=True, inferSchema=True)

# Select and rename columns
df = df.select(col("Date").alias("date"), col("Adj Close").alias("price_t"))

# Calculate returns (using a window function)
df = df.withColumn("returns", expr("price_t / lag(price_t, 1) over (order by date) - 1"))

# Calculate variance of returns
var_fb = df.selectExpr("variance(returns)").collect()[0][0]

# Calculate standard deviation (total risk)
sd_fb = var_fb ** 0.5

# Calculate deviations
df = df.withColumn("deviations", expr("returns - avg(returns) over ()"))

# Calculate squared deviations
df = df.withColumn("squared_deviations", expr("deviations * deviations"))

# Sum of squared deviations
sum_squared_deviations = df.selectExpr("sum(squared_deviations)").collect()[0][0]

# Estimate variance manually
var_fb_manual = sum_squared_deviations / (df.count() - 1)

# Annualized standard deviation
sd_fb_annual = sd_fb * (250 ** 0.5)

# Annualized expected return (crude method)
annual_return_crude = df.selectExpr("avg(returns)").collect()[0][0] * 250

# Annualized expected return (sophisticated method)
annual_return_sophisticated = (1 + df.selectExpr("avg(returns)").collect()[0][0]) ** 250 - 1

# Display results
print("Variance of Facebook's returns:", var_fb)
print("Standard Deviation (Total Risk) of Facebook's returns:", sd_fb)
print("Annualized Standard Deviation:", sd_fb_annual)
print("Annualized Expected Return (Crude):", annual_return_crude)
print("Annualized Expected Return (Sophisticated):", annual_return_sophisticated)



spark.stop()


Variance of Facebook's returns: 0.0005468017515752951
Standard Deviation (Total Risk) of Facebook's returns: 0.023383792497695816
Annualized Standard Deviation: 0.36973022312738213
Annualized Expected Return (Crude): 0.3373654646875022
Annualized Expected Return (Sophisticated): 0.40093243279785296
